In [13]:
#!cp /opt/workspace/msc_project/other/vtu_files/smooth_potato_20.vtu /mnt/c/Users/yoavr/Desktop/smooth_potato_20.vtu

In [14]:
import xml.etree.ElementTree as ET
import numpy as np

In [29]:
vtu_file = "/mnt/c/Users/yoavr/Desktop/paraview_pipeline/tests/round_100_neg.vtu"
tree = ET.parse(vtu_file)
num_positive_c = 0
c_positive = 0.8
num_negative_c = 100
c_negative = -0.8
root = tree.getroot()

In [30]:
# display the structure of the xml tree
for child in root:
    print(child.tag, child.attrib)
print(r'###')
spon_curv = root.findall('.//*[@Name="spontaneous_curvature"]')[0]
#for child in root[4][0][0]:
#    print(child.tag, child.attrib)
spon_curv.tag, spon_curv.attrib

dumpdate {}
tape {}
trisurf {'nvtx': '4502', 'npoly': '0', 'nmono': '0', 'compressed': 'false', 'seed': '1618847490'}
UnstructuredGrid {}
###


('DataArray',
 {'type': 'Float64', 'Name': 'spontaneous_curvature', 'format': 'ascii'})

In [31]:
# spon_curv = root[4][0][0][1] # get the node of spontaneous curvature
# more properly: 
spon_curv = root.findall('.//*[@Name="spontaneous_curvature"]')[0]

In [32]:
curvatures_txt = spon_curv.text.split(' ')[:-1]
print(curvatures_txt[:5],"...")

# more properly, an internal function from ts_vtu_python.py: 
def _xml_to_str_list(node):
    r"""Take xml node, extract text array to list.

    Convert sequence to 1d array and lines to 2d arrays, e.g
     2 3\n4 5 6\n' -> [['2','3'],['4','5','6']]
    '\n 2 3 5' -> ['2','3','5']
    """
    all_text = node.text.strip()

    # if there are tabs in the middle, assume this is a 2d list
    if all_text.__contains__('\n'):
        return [x.split() for x in all_text.split('\n')]
    else:  # no tabs: 1d array
        return all_text.split()

# they give the same answer
curvatures_txt == _xml_to_str_list(spon_curv)

['0.00000000000000000e+00', '0.00000000000000000e+00', '0.00000000000000000e+00', '0.00000000000000000e+00', '0.00000000000000000e+00'] ...


True

In [33]:
curvatures = list(map(float,curvatures_txt)) # convert to float
cs = np.array(curvatures)
print(cs[:10],"...")

[0.  0.  0.  0.  0.  0.  0.  0.  0.8 0. ] ...


In [34]:
# find bare vertices to change to cmc ones
# (as to not overwrite any existing cmc-structure)
cs_zero_idxs = cs == 0 # boolean indices
cs_zero_idxs_numberal = np.arange(len(cs))[cs_zero_idxs] # to number indices
print(cs_zero_idxs_numberal[:5],"...")

[0 1 2 3 4] ...


In [35]:
# choose some of the indices to change
idx_to_change = np.random.default_rng().choice(len(cs_zero_idxs_numberal), 
                                               num_positive_c+num_negative_c,
                                               replace=False) # yep: indices of indices
zero_idxs_to_change = cs_zero_idxs_numberal[idx_to_change]
print("first 5 random indices to change:\n",zero_idxs_to_change[:5],
      "\ncurvature at indices\n", cs[zero_idxs_to_change[:5]])

first 5 random indices to change:
 [3960 2251 2737 2250 2457] 
curvature at indices
 [0. 0. 0. 0. 0.]


In [37]:
sum(cs==-0.8)

100

In [90]:
cs[zero_idxs_to_change[:num_positive_c]] = c_positive
cs[zero_idxs_to_change[num_positive_c:]] = c_negative

In [91]:
# lousy way 
new_spon_curv=''
for number in cs:
    new_spon_curv += f'{number:.17e} '

# better way
new_spon_curv_2 = ''.join(f'{number:.17e} ' for number in cs)
new_spon_curv == new_spon_curv_2

True

In [92]:
from_c, to_c = 500, 1500
print(f"old text [{from_c}:{to_c}]:")
print(spon_curv.text[from_c:to_c])
print(f'\nnew text: [{from_c}:{to_c}]')
print(new_spon_curv[from_c:to_c]) # peek at data

old text [500:1500]:
+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 8.00000000000000044e-01 0.00000000000000000e+00 0.00000000000000000e+00 8.00000000000000044e-01 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 8.00000000000000044e-01 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 8.00000000000000044e-01 0.0000000000000

In [93]:
spon_curv.text = new_spon_curv # update curvature text

In [94]:
#see that the change applied
root2=tree.getroot()
print(root2.findall('.//*[@Name="spontaneous_curvature"]')[0].text.split(' ')[10:30])

['8.00000000000000044e-01', '0.00000000000000000e+00', '0.00000000000000000e+00', '0.00000000000000000e+00', '0.00000000000000000e+00', '0.00000000000000000e+00', '8.00000000000000044e-01', '0.00000000000000000e+00', '0.00000000000000000e+00', '0.00000000000000000e+00', '0.00000000000000000e+00', '0.00000000000000000e+00', '0.00000000000000000e+00', '0.00000000000000000e+00', '0.00000000000000000e+00', '-8.00000000000000044e-01', '8.00000000000000044e-01', '0.00000000000000000e+00', '0.00000000000000000e+00', '8.00000000000000044e-01']


In [95]:
#!!! uncomment to modify the vtu file!!!
#tree.write(r'/mnt/c/Users/yoavr/Desktop/paraview_pipeline/tests/round_100_neg.vtu',encoding=r'UTF-8')